In [50]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from praenomen2genus import praenomen2genus
genie = praenomen2genus.Genie()
import pandas as pd

genie.get_gender("Eirik")
#(u'Eirik', 'man', 'list_lookup')
genie.get_gender("Ulerikka")
#(u'Ulerikka', 'kvinne', 'predictor')
genie.get_gender("Ellen")

('Ellen', 'kvinne', 'list_lookup')

In [288]:
# Har navnedag
url = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager"
data = pd.read_html(url, header=0, index_col=0)[0]
#data.head()
data = data[["Navn"]]
data.dropna(subset=['Navn'], how='all', inplace = True) # skip 1.jan, er nan
data.head()

# | Navn  | dato
# | Eirik | 18 mai
har_navnedag = []
har_navnedag.append(["Jesus", "1. januar"])

rene_navn1 = []

for d in data.iterrows():
    #print(d[0], type(d[0]))
    #print(d[1].Navn)
    navn_pr_dag = [n for n in d[1].Navn.replace(",","").split() if n.lower() not in ["og", "(navn)"]]
    for n in navn_pr_dag:
        har_navnedag.append([n, d[0]])
        rene_navn1.append(n)
    #print()

df = pd.DataFrame.from_records(har_navnedag, columns=["Navn", "Dato"])
# save only this one scrape
#df.to_csv(path_or_buf="praenomen2genus/data/har_norsk_navnedag.csv", sep='\t', encoding='utf-8', index=False)

df.set_index("Navn", inplace=True)
#df.head()


#    !

# pokker finnes enda en liste:
url2 = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager_sortert_etter_navn"
data2 = pd.read_html(url2, header=0, index_col=0)[0]
data2 = data2[["Navnedag"]]
data2.columns = [["Dato"]]
#data2.head()
#data.describe()
rene_navn2 = []
for nn in data2.iterrows():
    rene_navn2.append(nn[0])

bare_i_forste = [n for n in rene_navn1 if n not in rene_navn2]
bare_i_andre = [n for n in rene_navn2 if n not in rene_navn1]

#print(len(bare_i_forste), bare_i_forste)
#print(len(bare_i_andre), bare_i_andre)
print("jeg burde ha så mange navn:", len(set(rene_navn1+rene_navn2)))


# la oss merge disse

#result = df.append(data2) #ignore_index=True # blir 1662 rader, virker for mye...
#result.describe()
#result

result2 = pd.merge(df, data2, how="outer", right_index=True, left_index=True) # 

# clean up multiple datoer...
result2["dato"] = result2["Dato_x"].fillna(result2["Dato_y"]) # trick!
result2 = result2[["dato"]] # clean date please
result2.head()



len(result2)

# lagre...
#result2.to_csv(path_or_buf="praenomen2genus/data/har_norsk_navnedag.csv", sep='\t', encoding='utf-8')#, index=False


,Navn
Dato,
2. januar,Dagfinn og Dagfrid
3. januar,Alfred og Alf og Alva
4. januar,Roar og Roger
5. januar,Hanna og Hanne
6. januar,"Aslaug, Åslaug og Aisha"


jeg burde ha så mange navn: 854


,dato
Navn,
Abraham,20. desember
Ada,4. mars
Adam,24. desember
Adrian,4. mars
Agate,5. februar


855

In [295]:
begge_kjonn = ["Gunnleiv", "Kim"]
kvinne = ["Hallgjerd"]
menn = ["Klement", "Clement", "Clément", "Salve"]

har_navnedag_kjonn = []
for row in result2.itertuples():
    if genie.get_gender(row.Index)[2] != "list_lookup":
        if 
        print(row.Index, genie.get_gender(row.Index))
    else:
        gender = gender_string(genie.get_gender(row.Index)[2])
    
    har_navnedag_kjonn.append(gender)

har_navnedag = result2

Agate ('Agate', 'kvinne', 'predictor')
André ('André', 'man', 'predictor')
Bo ('Bo', 'man', 'predictor')
Botolv ('Botolv', 'man', 'predictor')
Eli ('Eli', 'kvinne', 'predictor')
Gislaug ('Gislaug', 'kvinne', 'predictor')
Gunnleiv ('Gunnleiv', 'man', 'predictor')
Hallgjerd ('Hallgjerd', 'man', 'predictor')
Heid ('Heid', 'kvinne', 'predictor')
Hildebjørg ('Hildebjørg', 'kvinne', 'predictor')
Kim ('Kim', 'man', 'predictor')
Klement ('Klement', 'man', 'predictor')
Morgan ('Morgan', 'man', 'predictor')
Märtha ('Märtha', 'kvinne', 'predictor')
Reinhard ('Reinhard', 'man', 'predictor')
Riborg ('Riborg', 'kvinne', 'predictor')
Robin ('Robin', 'man', 'predictor')
Rudi ('Rudi', 'man', 'predictor')
Salve ('Salve', 'man', 'predictor')
Svanaug ('Svanaug', 'kvinne', 'predictor')
Sylfest ('Sylfest', 'man', 'predictor')
Åste ('Åste', 'kvinne', 'predictor')


In [255]:
# har vært på stortinget
from bs4 import BeautifulSoup
import requests
import sys


def gender_string(inn):
    if inn == 'mann':
        return 'm'
    elif inn == 'man':
        return 'm'
    elif inn == 'kvinne':
        return 'f'
    else:
        return 'unknown'

perioder = [] # trenger for å hene folk...
perioder_url = "https://data.stortinget.no/eksport/stortingsperioder"

r = requests.get(perioder_url)
soup = BeautifulSoup(r.content, "lxml")

for per in soup.find_all('stortingsperiode'): #Would return all the <asn></asn> tags found!
    perioder.append(per.id.string)

# nå kan vi hente folk pr perioder..
folk = []
# bedre_folk = [Eneste_fornavn, fullt navn, kjønn]
repr_base_url = "http://data.stortinget.no/eksport/representanter?stortingsperiodeid="#"2009-2013"

#perioder = ["2009-2013"]
for periode in perioder:
    #print(repr_base_url+periode)#http://data.stortinget.no/eksport/representanter?stortingsperiodeid=2009-2013
    r = requests.get(repr_base_url+periode)
    suppe = BeautifulSoup(r.content, "lxml")
    #suppe
    for rep in suppe.find_all("representant"):
        clean_fname = rep.fornavn.string.split()[0]
        folk.append((clean_fname,rep.fornavn.string + " " + rep.etternavn.string, gender_string(rep.kjoenn.string)))

#folk
len(folk)
len(set(folk)) #sorted(set(folk))
uniqe_fnames = list(set([n[0] for n in set(folk)]))
len(uniqe_fnames)

# lagre stortingsrepresentantene
rep_df = pd.DataFrame.from_records(folk)
rep_df.columns = ["Fornvan", "Fullt navn", "kjønn"]
rep_df.head()
#rep_df.to_csv(path_or_buf="praenomen2genus/data/stortingsrepresentantnavn.csv", sep='\t', encoding='utf-8', index=False)#

stortingsrepresentantnavn = rep_df

2839

1141

451

,Fornvan,Fullt navn,kjønn
0,Abid,Abid Q. Raja,m
1,Anna,Anna Ljunggren,f
2,Audun,Audun Lysbakken,m
3,Anders,Anders Anundsen,m
4,Anniken,Anniken Huitfeldt,f


In [220]:
# norrøne navn
norr_url = "https://no.wikipedia.org/wiki/Liste_over_personer_i_norr%C3%B8n_mytologi"

norr = pd.read_html(norr_url, header=0, index_col=0)[0]
norr2 = norr[["Norrønt navn"]]

norr2.head()
#norr2.to_csv(path_or_buf="praenomen2genus/data/personer_i_norron_mytologi.csv", sep='\t', encoding='utf-8')#
personer_i_norron_mytologi = norr2

,Norrønt navn
Navn,
Allvis,"Allvís, «den som vet alt»"
Andrimne,"Andrimner, «en med sot i ansiktet»"
Andvare,Andvari
Angerboda,"Angrboða, «sorgvarsleren»"
Aurboda,Eirboða


In [254]:
# Bibelske navn
url1 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-02.html" # jenter
url2 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-01.html" # gutter

bibjent = pd.read_html(url1, index_col=0)[1] #  header=0, føkket det til..
bibjent.columns = [["Kommentar"]]
bibjent.index.names = ['Navn']
bibjent.head()

bibeljenter = ['Abiah', 'Abigail', 'Abijah', 'Abilene', 'Abishag', 'Abital', 'Adah', 'Adina', 'Apphia', 'Ariel', 'Asenath', 'Ashtoreth', 'Atarah', 'Azubah', 'Basemath', 'Bashemath', 'Basmath', 'Bathsheba', 'Bernice', 'Bethany', 'Bethel', 'Beulah', 'Bilhah', 'Bithiah', 'Candace', 'Carmel', 'Chloe', 'Damaris', 'Deborah', 'Delilah', 'Dinah', 'Dorcas', 'Drusilla', 'Eden', 'Edna', 'Elisabeth', 'Elisheba', 'Elizabeth', 'Esther', 'Eunice', 'Eve', 'Gethsemane', 'Hadassah', 'Hagar', 'Hannah', 'Havilah', 'Helah', 'Hephzibah', 'Hepzibah', 'Hulda', 'Huldah', 'Ivah', 'Jael', 'Jedidah', 'Jemima', 'Jerusha', 'Jezebel', 'Joanna', 'Jordan', 'Judith', 'Julia', 'Keren-happuch', 'Keturah', 'Keziah', 'Leah', 'Lois', 'Lydia', 'Magdalene', 'Mahalath', 'Mahlah', 'Mara', 'Martha', 'Mary', 'Mehetabel', 'Mehitabel', 'Micaiah', 'Michal', 'Miriam', 'Moriah', 'Naamah', 'Naomi', 'Neriah', 'Noa', 'Noah', 'Ophrah', 'Orpah', 'Peninnah', 'Persis', 'Philadelphia', 'Phoebe', 'Prisca', 'Priscilla', 'Rachel', 'Rahab', 'Rebecca', 'Rebekah', 'Rhoda', 'Ruth', 'Salome', 'Sapphira', 'Sarah', 'Sarai', 'Sela', 'Sharon', 'Sherah', 'Shiphrah', 'Susanna', 'Susannah', 'Syntyche', 'Tabitha', 'Talitha', 'Tamar', 'Tirzah', 'Tryphena', 'Tryphosa', 'Vashti', 'Zillah', 'Zilpah', 'Zipporah']
for j in bibeljenter:
    if j not in bibjent.index:
        bibjent.loc[j] = pd.Series({'Kommentar': "fra liste"})

bibjent["Kjønn"] = "f"
bibjent.tail()


bibgutt = pd.read_html(url2, header=0, index_col=0)[1]
bibgutt.columns = [["Kommentar"]]
bibgutt.index.names = ['Navn']
bibgutt["Kjønn"] = "m"
bibgutt.head()

,Kommentar
Navn,
Ada,En av Esaus koner.
Anna,Profetinne i tempelet da Jesus ble født; Jesu ...
Batseba,"Urias kone, så kong Davids. Mor til kong Salomo."
Debora,Modig profet og dommer for israelsfoket ca år ...
Dina,Datter av Jacob og Lea.


,Kommentar,Kjønn
Navn,,
Tryphosa,fra liste,f
Vashti,fra liste,f
Zillah,fra liste,f
Zilpah,fra liste,f
Zipporah,fra liste,f


,Kommentar,Kjønn
Navn,,
Abel,Sønn av Adam og Eva. Slått i hjel av sin bror ...,m
Abraham,Israelfolkets stamfar.,m
Assalon,Sønn av kong David.,m
Adam,Den første mann.,m
Amos,Profet ca. 750 f.Kr.,m


In [171]:
# Jødiske navn

In [173]:
# Populære navn (periodisk oppdatet fra ssb?)
# for mange land her: http://www.uib.no/lle/23822/navnestatistikk-fra-mange-land

In [ ]:
# Som rimer på
# som likner på
# som inneholder en "o"
# som besteforeldrenes generasjon brukte
# som olderforeldrenes generasjon brukte
# som tip-tip-tip...


In [213]:
from pandas.io import sql
import pymysql
def connect():
    try:
        rdbms_username = 'root'#settings['user']
        rdbms_password = 'root'#settings['password']
        connection = pymysql.connect(
            unix_socket="/Applications/MAMP/tmp/mysql/mysql.sock",
            user=rdbms_username,
            passwd=rdbms_password,
            db="navn", charset='utf8',
            autocommit=True)
        cur = connection.cursor()
        cur.execute("USE navn;")
        return connection, cur
    except:
        print("kunne ikke logge på databasen")


def disconnect(connection):
    if connection:
        connection.close()
        print("koblet av mysql")

mysql_connection, mysql_cur = connect()
#sql = '''SELECT CONCAT(title, " ", full_text) as text, url, man_clas FROM
#nrk2013b_man_vs_mac_tbl WHERE man_clas IS NOT NULL'''
#mysql_cur.execute(sql)
#rows = mysql_cur.fetchall()


In [284]:
# lagre i db

def df2mysql(df, sql_table_name, gender_row="Kjønn"):
    mysql_connection, mysql_cur = connect()
    #df.to_sql(sql_table_name, mysql_connection, flavor=None, schema=None, if_exists='fail', index=True, index_label=index_label, chunksize=None, dtype=None)
    for row in df.iterrows():
        #print(row[0], row[1]["Kjønn"])
        #print()
        sql = "REPLACE INTO `{0}` (`name`, `gender`) VALUES ('{1}', '{2}');".format(sql_table_name, row[0], row[1][gender_row])        #print(sql)

        mysql_cur.execute(sql);
    #mysql_cur.commit()

#bibjent.head()
#bibgutt.head()
#df2mysql(bibjent, sql_table_name="bibelske", gender_row="Kjønn")    
#df2mysql(bibgutt, sql_table_name="bibelske", gender_row="Kjønn")    

har_navnedag.head()
#df2mysql(bibgutt, sql_table_name="bibelske", gender_row="Kjønn")    


stortingsrepresentantnavn.head()
personer_i_norron_mytologi.head()


,dato
Navn,
Abraham,20. desember
Ada,4. mars
Adam,24. desember
Adrian,4. mars
Agate,5. februar


,Fornvan,Fullt navn,kjønn
0,Abid,Abid Q. Raja,m
1,Anna,Anna Ljunggren,f
2,Audun,Audun Lysbakken,m
3,Anders,Anders Anundsen,m
4,Anniken,Anniken Huitfeldt,f


,Norrønt navn
Navn,
Allvis,"Allvís, «den som vet alt»"
Andrimne,"Andrimner, «en med sot i ansiktet»"
Andvare,Andvari
Angerboda,"Angrboða, «sorgvarsleren»"
Aurboda,Eirboða
